# AutoGen Werkzeugnutzungsbeispiel


## Importieren Sie die benötigten Pakete


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Erstellen des Clients

In diesem Beispiel verwenden wir [GitHub Models](https://aka.ms/ai-agents-beginners/github-models), um Zugriff auf das LLM zu erhalten.

Das `model` ist als `gpt-4o-mini` definiert. Versuchen Sie, das Modell auf ein anderes im GitHub Models-Marktplatz verfügbares Modell zu ändern, um unterschiedliche Ergebnisse zu sehen.

Als schnellen Test führen wir einfach eine einfache Eingabeaufforderung aus – `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definition der Funktionen

In diesem Beispiel geben wir dem Agenten Zugriff auf ein Tool, das eine Funktion mit einer Liste verfügbarer Reiseziele und deren Verfügbarkeit ist.

Man kann sich vorstellen, dass dies ein Szenario wäre, in dem ein Reisebüro beispielsweise Zugriff auf eine Reisedatenbank hat.

Während Sie dieses Beispiel durchgehen, können Sie gerne neue Funktionen und Tools definieren, die der Agent aufrufen kann.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definition des Function Tools
Damit der Agent `vacation_destinations` als `FunctionTool` verwenden kann, müssen wir es als solches definieren.

Wir werden außerdem eine Beschreibung des Tools bereitstellen, die dem Agenten hilft, zu verstehen, wofür dieses Tool im Zusammenhang mit der vom Benutzer angeforderten Aufgabe verwendet wird.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definition des Agenten

Nun können wir den Agenten im folgenden Code erstellen. Wir definieren die `system_message`, um dem Agenten Anweisungen zu geben, wie er Nutzern bei der Suche nach Urlaubszielen helfen kann.

Außerdem setzen wir den Parameter `reflect_on_tool_use` auf true. Dadurch kann das LLM die Antwort des Tool-Aufrufs nehmen und die Antwort in natürlicher Sprache weitergeben.

Sie können den Parameter auf false setzen, um den Unterschied zu sehen.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Ausführen des Agenten

Jetzt können wir den Agenten mit der anfänglichen Benutzeranfrage starten, in der nach einer Reise nach Tokio gefragt wird.

Sie können dieses Städteziel ändern, um zu sehen, wie der Agent auf die Verfügbarkeit der Stadt reagiert.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
